# Importamos las librerías necesarias para el manejo del dataset, además de la librería sklearn.

In [ ]:
import pandas as pd
import numpy as np
import sklearn as skl

# Se carga y se visualiza el archivo train.csv.

In [ ]:
df = pd.read_csv('data/train.csv')
df

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Postgrado,No,medio-alto
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Técnica o tecnológica incompleta,No,bajo
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No,bajo
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Secundaria (Bachillerato) completa,No,alto
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Primaria completa,No,medio-bajo
...,...,...,...,...,...,...,...,...,...,...,...,...
692495,25096,20195,BIOLOGIA,LA GUAJIRA,Entre 500 mil y menos de 1 millón,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) incompleta,Si,medio-alto
692496,754213,20212,PSICOLOGIA,NORTE SANTANDER,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Primaria incompleta,Secundaria (Bachillerato) incompleta,No,bajo
692497,504185,20183,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,Menos de 10 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) incompleta,Si,medio-bajo
692498,986620,20195,PSICOLOGIA,TOLIMA,Entre 2.5 millones y menos de 4 millones,Menos de 10 horas,Estrato 1,No,Primaria completa,Primaria completa,Si,bajo


# Se define la función que convertirá los valores de la columna ESTU_VALORMATRICULAUNIVERSIDAD a valores numéricos. Será un valaor promedio entre el límite inferior y superior.

In [ ]:
def convertir_valor_matricula(valor):
    if pd.isna(valor):
        return np.nan
    elif 'Menos de 500 mil' in valor:
        return 250000  # Asignamos un valor promedio entre 0 y 500 mil
    elif 'Entre 500 mil y menos de 1 millón' in valor:
        return 750000  # Promedio entre 500 mil y 1 millón
    elif 'Entre 1 millón y menos de 2.5 millones' in valor:
        return 1750000  # Promedio entre 1 y 2.5 millones
    elif 'Entre 2.5 millones y menos de 4 millones' in valor:
        return 3250000  # Promedio entre 2.5 y 4 millones
    elif 'Entre 4 millones y menos de 5.5 millones' in valor:
        return 4750000  # Promedio entre 4 y 5.5 millones
    elif 'Entre 5.5 millones y menos de 7 millones' in valor:
        return 6250000  # Promedio entre 5.5 y 7 millones
    elif 'Más de 7 millones' in valor:
        return 7500000  # Asignamos un valor mínimo representativo superior a 7 millones
    elif 'No pagó matrícula' in valor:
        return 0  # Asumimos que no se pagó nada
    else:
        return np.nan  # Para cualquier caso que no coincida

# Creamos una copia del df y luego aplicamos la función convertir_valor_matricula a toda la columna de ESTU_VALORMATRICULAUNIVERSIDAD. Por otro lado, cuando todos los valores estén convertidos en números, se reemplazarán los nalores nulos con la media de los valores de las columna.

In [ ]:
df_copy = df.copy()
df_copy['ESTU_VALORMATRICULAUNIVERSIDAD'] = df_copy['ESTU_VALORMATRICULAUNIVERSIDAD'].apply(convertir_valor_matricula)

media = df_copy ['ESTU_VALORMATRICULAUNIVERSIDAD'].mean()
df_copy['ESTU_VALORMATRICULAUNIVERSIDAD'] = df_copy['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna(media.round(3))

df_copy['ESTU_VALORMATRICULAUNIVERSIDAD'].unique() # Se verifica que ya no queden valores nulos en la columna ESTU_VALORMATRICULAUNIVERSIDAD

array([6250000.   , 3250000.   , 4750000.   , 7500000.   , 1750000.   ,
        750000.   ,  250000.   ,       0.   , 2815800.269])

# Se define la función para convertir los valores de ESTU_HORASSEMANATRABAJA en datos numéricos.

In [ ]:
def convertir_horas_trabajadas(valor):
    if isinstance(valor, str):
        if "Entre" in valor:
            partes = valor.split('y')
            min_val = float(partes[0].split(' ')[-2])  # Obtener el penúltimo elemento
            max_val = float(partes[1].split(' ')[-2])  # Obtener el penúltimo elemento de la segunda parte
            return (min_val + max_val) / 2  # Retornar el promedio del rango

        elif "Más de" in valor:
            return float(valor.split(' ')[2])  # Convertir a número

        elif "Menos de" in valor:
            return float(valor.split(' ')[2])  # Convertir a número

        elif "0" in valor:
            return 0  # Devolver 0 en número

    return np.nan  # Devolver NaN si no es un valor válido

# Se aplica la función convertir_horas_trabajadas a toda la columna ESTU_HORASSEMANATRABAJA. Por otro lado, los valores nulos se reemplazan por 0.

In [ ]:
df_copy['ESTU_HORASSEMANATRABAJA'] = df_copy['ESTU_HORASSEMANATRABAJA'].apply(convertir_horas_trabajadas)

df_copy['ESTU_HORASSEMANATRABAJA'] = df_copy['ESTU_HORASSEMANATRABAJA'].fillna(0)

df_copy['ESTU_HORASSEMANATRABAJA'].unique() # Se verifica que ya no queden valores nulos en la columna ESTU_HORASSEMANATRABAJA

array([10. ,  0. , 30. , 25.5, 15.5])

# Se crea un diccionario para asignar el valor numérico a cada valor en letra de la columna FAMI_ESTRATOVIVIENDA. Posteriormente se reemplazan en el df los estratos por el número correspondiente utilizando el diccionario creado.

In [ ]:
estrato_dict = {
    'Estrato 1': 1,
    'Estrato 2': 2,
    'Estrato 3': 3,
    'Estrato 4': 4,
    'Estrato 5': 5,
    'Estrato 6': 6,
    'Sin Estrato': 0,
    np.nan: 0  # Cambiar nan a 0
}

df_copy['FAMI_ESTRATOVIVIENDA'] = df_copy['FAMI_ESTRATOVIVIENDA'].replace(estrato_dict)

df_copy['FAMI_ESTRATOVIVIENDA'].unique() # Se verifica que ya no queden valores nulos en la columna FAMI_ESTRATOVIVIENDA

<ipython-input-41-7a2971612dd2>:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_copy['FAMI_ESTRATOVIVIENDA'] = df_copy['FAMI_ESTRATOVIVIENDA'].replace(estrato_dict)


array([3., 4., 5., 2., 1., 0., 6.])

# Se reemplazan los valores de la columna ESTU_PAGOMATRICULAPROPIO a datos numéricos, "Si" por 1 y "No" por 0. Además, se reemplazan los valores nulos por 0.

In [ ]:
df_copy['ESTU_PAGOMATRICULAPROPIO'] = df_copy['ESTU_PAGOMATRICULAPROPIO'].replace({'Si': 1, 'No': 0})

df_copy['ESTU_PAGOMATRICULAPROPIO'] = df_copy['ESTU_PAGOMATRICULAPROPIO'].fillna(0)

df_copy['ESTU_PAGOMATRICULAPROPIO'].unique() # Se verifica que ya no queden valores nulos en la columna FAMI_TIENEINTERNET

<ipython-input-43-add613f659cb>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_copy['ESTU_PAGOMATRICULAPROPIO'] = df_copy['ESTU_PAGOMATRICULAPROPIO'].replace({'Si': 1, 'No': 0})


array([0., 1.])

# Se crea un diccionario para asignar el valor numérico a cada valor en letra de la columna RENDIMIENTO_GLOBAL de 1 a 4, de más bajo a más alto.

In [ ]:
rend_dict = {
    'bajo': 1,
    'medio-bajo': 2,
    'medio-alto': 3,
    'alto': 4,
}

df_copy['RENDIMIENTO_GLOBAL'] = df_copy['RENDIMIENTO_GLOBAL'].replace(rend_dict)

df_copy['RENDIMIENTO_GLOBAL'].unique() # Se verifica que ya no queden valores nulos en la columna RENDIMIENTO_GLOBAL

<ipython-input-44-0a918cfde127>:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_copy['RENDIMIENTO_GLOBAL'] = df_copy['RENDIMIENTO_GLOBAL'].replace(rend_dict)


array([3, 1, 4, 2])

# Se reemplazan los valores de la columna FAMI_TIENEINTERNET a datos numéricos, "Si" por 1 y "No" por 0. Además, se reemplazan los valores nulos por 0.

In [ ]:
df_copy['FAMI_TIENEINTERNET'] = df_copy['FAMI_TIENEINTERNET'].replace({'Si': 1, 'No': 0})

df_copy['FAMI_TIENEINTERNET'] = df_copy['FAMI_TIENEINTERNET'].fillna(0)

df_copy['FAMI_TIENEINTERNET'].unique() # Se verifica que ya no queden valores nulos en la columna FAMI_TIENEINTERNET

<ipython-input-42-7843fa38d5da>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_copy['FAMI_TIENEINTERNET'] = df_copy['FAMI_TIENEINTERNET'].replace({'Si': 1, 'No': 0})


array([1., 0.])

# Se tomó la función vista en el NoteBook 04.02 de las clases, para convertir valaores de columnas a one hot.

In [ ]:
def to_onehot(x):
    values = np.unique(x)
    r = np.r_[[np.argwhere(i==values)[0][0] for i in x]]
    return np.eye(len(values))[r].astype(int)

def replace_column_with_onehot(d, col):
    assert sum(d[col].isna())==0, "column must have no NaN values"
    values = np.unique(d[col]
                      )
    k = to_onehot(d[col].values)
    r = pd.DataFrame(k, columns=["%s_%s"%(col, values[i]) for i in range(k.shape[1])], index=d.index).join(d)
    del(r[col])
    return r

# Convertimos los valores de FAMI_EDUCACIONMADRE y FAMI_EDUCACIONPADRE en one hot usando la función anterior. Al final mostramos el df.

In [ ]:
col_m = 'FAMI_EDUCACIONMADRE'
df_copy[col_m].fillna('No Aplica', inplace=True)
madre_onehot = replace_column_with_onehot(df_copy[[col_m]], col_m)
df_copy = df_copy.join(madre_onehot)
df_copy = df_copy.drop(col_m, axis=1)

col_p = 'FAMI_EDUCACIONPADRE'
df_copy[col_p].fillna('No Aplica', inplace=True)
padre_onehot = replace_column_with_onehot(df_copy[[col_p]], col_p)
df_copy = df_copy.join(padre_onehot)
df_copy = df_copy.drop(col_p, axis=1)

df_copy

<ipython-input-46-0a93748ce821>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_copy[col_m].fillna('No Aplica', inplace=True)
<ipython-input-46-0a93748ce821>:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usin

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL,...,FAMI_EDUCACIONPADRE_Ninguno,FAMI_EDUCACIONPADRE_No Aplica,FAMI_EDUCACIONPADRE_No sabe,FAMI_EDUCACIONPADRE_Postgrado,FAMI_EDUCACIONPADRE_Primaria completa,FAMI_EDUCACIONPADRE_Primaria incompleta,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONPADRE_Técnica o tecnológica completa,FAMI_EDUCACIONPADRE_Técnica o tecnológica incompleta
0,904256,20212,ENFERMERIA,BOGOTÁ,6250000.0,10.0,3.0,1.0,0.0,3,...,0,0,0,0,0,0,0,0,0,1
1,645256,20212,DERECHO,ATLANTICO,3250000.0,0.0,3.0,0.0,0.0,1,...,0,0,0,0,0,0,0,0,1,0
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,3250000.0,30.0,3.0,1.0,0.0,1,...,0,0,0,0,0,0,1,0,0,0
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,4750000.0,0.0,4.0,1.0,0.0,4,...,0,0,1,0,0,0,0,0,0,0
4,989032,20212,PSICOLOGIA,ANTIOQUIA,3250000.0,25.5,3.0,1.0,0.0,2,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,25096,20195,BIOLOGIA,LA GUAJIRA,750000.0,15.5,2.0,1.0,1.0,3,...,0,0,0,0,0,0,1,0,0,0
692496,754213,20212,PSICOLOGIA,NORTE SANTANDER,3250000.0,30.0,3.0,1.0,0.0,1,...,0,0,0,0,0,1,0,0,0,0
692497,504185,20183,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,1750000.0,10.0,3.0,1.0,1.0,2,...,0,0,0,0,0,0,1,0,0,0
692498,986620,20195,PSICOLOGIA,TOLIMA,3250000.0,10.0,1.0,0.0,1.0,1,...,0,0,0,0,1,0,0,0,0,0


# Eliminamos las columnas de programa académico, departamento y el periodo, al final mostramos el df.

In [ ]:
df_copy = df_copy.drop(['ESTU_PRGM_ACADEMICO'], axis=1)
df_copy = df_copy.drop(['PERIODO'], axis=1)
df_copy = df_copy.drop(['ESTU_PRGM_DEPARTAMENTO'], axis=1)

df_copy

,ID,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL,FAMI_EDUCACIONMADRE_Educación profesional completa,FAMI_EDUCACIONMADRE_Educación profesional incompleta,FAMI_EDUCACIONMADRE_Ninguno,...,FAMI_EDUCACIONPADRE_Ninguno,FAMI_EDUCACIONPADRE_No Aplica,FAMI_EDUCACIONPADRE_No sabe,FAMI_EDUCACIONPADRE_Postgrado,FAMI_EDUCACIONPADRE_Primaria completa,FAMI_EDUCACIONPADRE_Primaria incompleta,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONPADRE_Técnica o tecnológica completa,FAMI_EDUCACIONPADRE_Técnica o tecnológica incompleta
0,904256,6250000.0,10.0,3.0,1.0,0.0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,645256,3250000.0,0.0,3.0,0.0,0.0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,308367,3250000.0,30.0,3.0,1.0,0.0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,470353,4750000.0,0.0,4.0,1.0,0.0,4,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,989032,3250000.0,25.5,3.0,1.0,0.0,2,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,25096,750000.0,15.5,2.0,1.0,1.0,3,0,0,0,...,0,0,0,0,0,0,1,0,0,0
692496,754213,3250000.0,30.0,3.0,1.0,0.0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
692497,504185,1750000.0,10.0,3.0,1.0,1.0,2,0,0,0,...,0,0,0,0,0,0,1,0,0,0
692498,986620,3250000.0,10.0,1.0,0.0,1.0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0


# Separamos las columnas que sometemos a evaluación en X, y la predición real en Y.

In [ ]:
X_train = df_copy.drop(['RENDIMIENTO_GLOBAL', 'ID'], axis=1)
y_train = df_copy['RENDIMIENTO_GLOBAL']

# Importamos los clasificadores desde sklearn, definimos el clasificador RandomForestClassifier y lo entrenamos con nuestros datos.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

model = Pipeline(steps=[('clasifier', RandomForestClassifier(random_state=42))])
model.fit(X_train, y_train)

Pipeline(steps=[('clasifier', RandomForestClassifier(random_state=42))])

# Importamos el accuracy_score de sklearn y calculamos el accuracy con los datos de train.

In [ ]:
from sklearn.metrics import accuracy_score

y_val_pred = model.predict(X_train)
accuracy = accuracy_score(y_train, y_val_pred)
print("Accuracy:", accuracy)

Accuracy: 0.45937617328519853


# Cargamos y mostramos el archivo test.csv.

In [ ]:
df_test = pd.read_csv('data/test.csv')
df_test

,Unnamed: 0,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO
0,0,550236,20183,TRABAJO SOCIAL,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Primaria completa,Si
1,1,98545,20203,ADMINISTRACION COMERCIAL Y DE MERCADEO,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Técnica o tecnológica completa,No
2,2,499179,20212,INGENIERIA MECATRONICA,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Secundaria (Bachillerato) completa,No
3,3,782980,20195,CONTADURIA PUBLICA,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Primaria incompleta,No
4,4,785185,20212,ADMINISTRACION DE EMPRESAS,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No
...,...,...,...,...,...,...,...,...,...,...,...,...
296781,296781,496981,20195,ADMINISTRACION DE EMPRESAS,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 1,Si,Primaria incompleta,Primaria incompleta,Si
296782,296782,209415,20183,DERECHO,META,Entre 1 millón y menos de 2.5 millones,0,Estrato 4,Si,Educación profesional completa,Educación profesional completa,No
296783,296783,239074,20212,DERECHO,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Educación profesional completa,No
296784,296784,963852,20195,INGENIERIA AERONAUTICA,ANTIOQUIA,Entre 5.5 millones y menos de 7 millones,Entre 11 y 20 horas,Estrato 3,Si,Educación profesional completa,Educación profesional completa,No


# Creamos una copia del df_test y luego aplicamos la función convertir_valor_matricula a toda la columna de ESTU_VALORMATRICULAUNIVERSIDAD. Por otro lado, cuando todos los valores estén convertidos en números, se reemplazarán los nalores nulos con la media de los valores de las columna.

In [ ]:
df_test_copy = df_test.copy()
df_test_copy['ESTU_VALORMATRICULAUNIVERSIDAD'] = df_test_copy['ESTU_VALORMATRICULAUNIVERSIDAD'].apply(convertir_valor_matricula)

media = df_test_copy ['ESTU_VALORMATRICULAUNIVERSIDAD'].mean()
df_test_copy['ESTU_VALORMATRICULAUNIVERSIDAD'] = df_test_copy['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna(media.round(3))

df_test_copy['ESTU_VALORMATRICULAUNIVERSIDAD'].unique() # Se verifica que ya no queden valores nulos en la columna ESTU_VALORMATRICULAUNIVERSIDAD

array([ 250000.   , 3250000.   , 1750000.   ,  750000.   , 7500000.   ,
       4750000.   , 6250000.   ,       0.   , 2817658.121])

# Se aplica la función convertir_horas_trabajadas a toda la columna ESTU_HORASSEMANATRABAJA. Por otro lado, los valores nulos se reemplazan por 0.

In [ ]:
df_test_copy['ESTU_HORASSEMANATRABAJA'] = df_test_copy['ESTU_HORASSEMANATRABAJA'].apply(convertir_horas_trabajadas)

df_test_copy['ESTU_HORASSEMANATRABAJA'] = df_test_copy['ESTU_HORASSEMANATRABAJA'].fillna(0)

df_test_copy['ESTU_HORASSEMANATRABAJA'].unique() # Se verifica que ya no queden valores nulos en la columna ESTU_HORASSEMANATRABAJA

array([10. , 25.5,  0. , 15.5, 30. ])

# Usando el mismo diccionario para el estrato, se le asigna el valor numérico a cada valor en letra de la columna FAMI_ESTRATOVIVIENDA.

In [ ]:
df_test_copy['FAMI_ESTRATOVIVIENDA'] = df_test_copy['FAMI_ESTRATOVIVIENDA'].replace(estrato_dict)

df_test_copy['FAMI_ESTRATOVIVIENDA'].unique() # Se verifica que ya no queden valores nulos en la columna FAMI_ESTRATOVIVIENDA

<ipython-input-54-003e6fd35edb>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_test_copy['FAMI_ESTRATOVIVIENDA'] = df_test_copy['FAMI_ESTRATOVIVIENDA'].replace(estrato_dict)


array([3., 2., 1., 4., 0., 6., 5.])

# Se reemplazan los valores de la columna FAMI_TIENEINTERNET a datos numéricos, "Si" por 1 y "No" por 0. Además, se reemplazan los valores nulos por 0.

In [ ]:
df_test_copy['FAMI_TIENEINTERNET'] = df_test_copy['FAMI_TIENEINTERNET'].replace({'Si': 1, 'No': 0})

df_test_copy['FAMI_TIENEINTERNET'] = df_test_copy['FAMI_TIENEINTERNET'].fillna(0)

df_test_copy['FAMI_TIENEINTERNET'].unique() # Se verifica que ya no queden valores nulos en la columna FAMI_TIENEINTERNET

<ipython-input-55-3cfa7971e3b6>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_test_copy['FAMI_TIENEINTERNET'] = df_test_copy['FAMI_TIENEINTERNET'].replace({'Si': 1, 'No': 0})


array([1., 0.])

# Se reemplazan los valores de la columna ESTU_PAGOMATRICULAPROPIO a datos numéricos, "Si" por 1 y "No" por 0. Además, se reemplazan los valores nulos por 0.

In [ ]:
df_test_copy['ESTU_PAGOMATRICULAPROPIO'] = df_test_copy['ESTU_PAGOMATRICULAPROPIO'].replace({'Si': 1, 'No': 0})

df_test_copy['ESTU_PAGOMATRICULAPROPIO'] = df_test_copy['ESTU_PAGOMATRICULAPROPIO'].fillna(0)

df_test_copy['ESTU_PAGOMATRICULAPROPIO'].unique() # Se verifica que ya no queden valores nulos en la columna FAMI_TIENEINTERNET

<ipython-input-56-a4e77673aaa8>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_test_copy['ESTU_PAGOMATRICULAPROPIO'] = df_test_copy['ESTU_PAGOMATRICULAPROPIO'].replace({'Si': 1, 'No': 0})


array([1., 0.])

# Convertimos los valores de FAMI_EDUCACIONMADRE y FAMI_EDUCACIONPADRE en one hot usando la función. Al final mostramos el df.

In [ ]:
col_m = 'FAMI_EDUCACIONMADRE'
df_test_copy[col_m].fillna('No Aplica', inplace=True)
madre_onehot = replace_column_with_onehot(df_test_copy[[col_m]], col_m)
df_test_copy = df_test_copy.join(madre_onehot)
df_test_copy = df_test_copy.drop(col_m, axis=1)

col_p = 'FAMI_EDUCACIONPADRE'
df_test_copy[col_p].fillna('No Aplica', inplace=True)
padre_onehot = replace_column_with_onehot(df_test_copy[[col_p]], col_p)
df_test_copy = df_test_copy.join(padre_onehot)
df_test_copy = df_test_copy.drop(col_p, axis=1)

df_test_copy

<ipython-input-57-1750c87a0d6b>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_test_copy[col_m].fillna('No Aplica', inplace=True)
<ipython-input-57-1750c87a0d6b>:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try

,Unnamed: 0,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,ESTU_PAGOMATRICULAPROPIO,...,FAMI_EDUCACIONPADRE_Ninguno,FAMI_EDUCACIONPADRE_No Aplica,FAMI_EDUCACIONPADRE_No sabe,FAMI_EDUCACIONPADRE_Postgrado,FAMI_EDUCACIONPADRE_Primaria completa,FAMI_EDUCACIONPADRE_Primaria incompleta,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONPADRE_Técnica o tecnológica completa,FAMI_EDUCACIONPADRE_Técnica o tecnológica incompleta
0,0,550236,20183,TRABAJO SOCIAL,BOLIVAR,250000.0,10.0,3.0,1.0,1.0,...,0,0,0,0,0,0,0,0,1,0
1,1,98545,20203,ADMINISTRACION COMERCIAL Y DE MERCADEO,ANTIOQUIA,3250000.0,25.5,2.0,1.0,0.0,...,0,0,0,0,0,0,1,0,0,0
2,2,499179,20212,INGENIERIA MECATRONICA,BOGOTÁ,1750000.0,0.0,3.0,1.0,0.0,...,0,0,0,0,0,0,0,1,0,0
3,3,782980,20195,CONTADURIA PUBLICA,SUCRE,1750000.0,25.5,1.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0
4,4,785185,20212,ADMINISTRACION DE EMPRESAS,ATLANTICO,3250000.0,15.5,2.0,1.0,0.0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296781,296781,496981,20195,ADMINISTRACION DE EMPRESAS,BOGOTÁ,3250000.0,30.0,1.0,1.0,1.0,...,0,0,0,0,0,1,0,0,0,0
296782,296782,209415,20183,DERECHO,META,1750000.0,0.0,4.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
296783,296783,239074,20212,DERECHO,BOGOTÁ,3250000.0,30.0,3.0,1.0,0.0,...,0,0,0,0,0,0,1,0,0,0
296784,296784,963852,20195,INGENIERIA AERONAUTICA,ANTIOQUIA,6250000.0,15.5,3.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0


# Eliminamos las columnas de programa académico, departamento y el periodo, al final mostramos el df.

In [ ]:
df_test_copy = df_test_copy.drop(['ESTU_PRGM_ACADEMICO'], axis=1)
df_test_copy = df_test_copy.drop(['PERIODO'], axis=1)
df_test_copy = df_test_copy.drop(['ESTU_PRGM_DEPARTAMENTO'], axis=1)

df_test_copy

,Unnamed: 0,ID,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,ESTU_PAGOMATRICULAPROPIO,FAMI_EDUCACIONMADRE_Educación profesional completa,FAMI_EDUCACIONMADRE_Educación profesional incompleta,FAMI_EDUCACIONMADRE_Ninguno,...,FAMI_EDUCACIONPADRE_Ninguno,FAMI_EDUCACIONPADRE_No Aplica,FAMI_EDUCACIONPADRE_No sabe,FAMI_EDUCACIONPADRE_Postgrado,FAMI_EDUCACIONPADRE_Primaria completa,FAMI_EDUCACIONPADRE_Primaria incompleta,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONPADRE_Técnica o tecnológica completa,FAMI_EDUCACIONPADRE_Técnica o tecnológica incompleta
0,0,550236,250000.0,10.0,3.0,1.0,1.0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,98545,3250000.0,25.5,2.0,1.0,0.0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,2,499179,1750000.0,0.0,3.0,1.0,0.0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,3,782980,1750000.0,25.5,1.0,0.0,0.0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,4,785185,3250000.0,15.5,2.0,1.0,0.0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296781,296781,496981,3250000.0,30.0,1.0,1.0,1.0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
296782,296782,209415,1750000.0,0.0,4.0,1.0,0.0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
296783,296783,239074,3250000.0,30.0,3.0,1.0,0.0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
296784,296784,963852,6250000.0,15.5,3.0,1.0,0.0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


# Eliminamos la columna Unnamed: 0 que aparecía en el df.

In [ ]:
df_test_copy = df_test_copy.drop(['Unnamed: 0'], axis=1)
df_test_copy

,ID,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,ESTU_PAGOMATRICULAPROPIO,FAMI_EDUCACIONMADRE_Educación profesional completa,FAMI_EDUCACIONMADRE_Educación profesional incompleta,FAMI_EDUCACIONMADRE_Ninguno,FAMI_EDUCACIONMADRE_No Aplica,...,FAMI_EDUCACIONPADRE_Ninguno,FAMI_EDUCACIONPADRE_No Aplica,FAMI_EDUCACIONPADRE_No sabe,FAMI_EDUCACIONPADRE_Postgrado,FAMI_EDUCACIONPADRE_Primaria completa,FAMI_EDUCACIONPADRE_Primaria incompleta,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONPADRE_Técnica o tecnológica completa,FAMI_EDUCACIONPADRE_Técnica o tecnológica incompleta
0,550236,250000.0,10.0,3.0,1.0,1.0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,98545,3250000.0,25.5,2.0,1.0,0.0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,499179,1750000.0,0.0,3.0,1.0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,782980,1750000.0,25.5,1.0,0.0,0.0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,785185,3250000.0,15.5,2.0,1.0,0.0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296781,496981,3250000.0,30.0,1.0,1.0,1.0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
296782,209415,1750000.0,0.0,4.0,1.0,0.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
296783,239074,3250000.0,30.0,3.0,1.0,0.0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
296784,963852,6250000.0,15.5,3.0,1.0,0.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Dividimos los datos en X y Y, pero en X eliminamos la columna ID ya que no aporta información necesaria para las predicciones.

In [ ]:
X_test = df_test_copy.drop(['ID'], axis=1)
Y_test_pred = model.predict(X_test)

# Convertimos el conjunto Y_test_pred a series para poder convertirlas a un archivo csv. Además, se crea un diccionario para asignar el valor en letra a cada valor numérico de la columna RENDIMIENTO_GLOBAL, ya que vendrá en números. Finalmente creamos el dataframe con columnas ID y RENDIMIENTO_GLOBAL y el df lo convertimos a csv.

In [ ]:
Y_test_pred_series = pd.Series(Y_test_pred)

rend_dict_rev = {
    1: 'bajo',
    2: 'medio-bajo',
    3: 'medio-alto',
    4: 'alto',
}

submission = pd.DataFrame({
    'ID': df_test['ID'],
    'RENDIMIENTO_GLOBAL': Y_test_pred_series.replace(rend_dict_rev)
})

submission.to_csv('submission.csv', index=False)